# Red neuronal

En el siguiente notebook se emplean redes neuronales como medida de prediccion

In [105]:
import sklearn
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix


## Carga de archivo y division

In [106]:
data = pd.read_csv('depression.csv')
data.head()

,Gender,Age,CGPA,Sleep Duration,Degree,Suicidal Thoughts,Work/Study Hours,Family History of Mental Illness,Depression
0,Male,33.0,8.97,5-6 hours,Undergraduate,Yes,3.0,No,1
1,Female,24.0,5.90,5-6 hours,Undergraduate,No,3.0,Yes,0
2,Male,31.0,7.03,Less than 5 hours,Undergraduate,No,9.0,Yes,0
3,Female,28.0,5.59,7-8 hours,Undergraduate,Yes,4.0,Yes,1
4,Female,25.0,8.13,5-6 hours,Postgraduate,Yes,1.0,No,0


In [107]:
# Determinamos los porcentajes a dividir
NumData = len(data)
cTrain = int (NumData*0.8)
cTest = NumData-cTrain
print(NumData,cTrain,cTest)

27901 22320 5581


In [108]:
train_data, test_data = sklearn.model_selection.train_test_split(data, train_size=cTrain, test_size=cTest)

In [109]:
train_data.shape

(22320, 9)

In [110]:
test_data.shape

(5581, 9)

## Identificación de variables numéricas y categóricas

In [111]:
# Variables númericas
num_attribs = ["Age", "CGPA", "Work/Study Hours"]

train_data[num_attribs].describe()

,Age,CGPA,Work/Study Hours
count,22320.000000,22320.000000,22320.000000
mean,25.841353,7.656553,7.161559
std,4.904506,1.469377,3.713383
min,18.000000,0.000000,0.000000
25%,21.000000,6.280000,4.000000
50%,26.000000,7.770000,8.000000
75%,30.000000,8.920000,10.000000
max,59.000000,10.000000,12.000000


In [112]:
# Variables categoricas

cat_attribs = ["Gender", "Sleep Duration", "Degree", "Family History of Mental Illness"]
cat_attribs_st = ["Suicidal Thoughts"]
train_data[cat_attribs].describe()

,Gender,Sleep Duration,Degree,Family History of Mental Illness
count,22320,22320,22320,22320
unique,2,5,5,2
top,Male,Less than 5 hours,Undergraduate,No
freq,12467,6674,10125,11468


## Construcción de los pipelines

In [113]:
# Pipeline para atributos numéricos

num_pipeline = Pipeline([
    # Sin imputer, como vimos en el EDA, no hay datos faltantes
    ("scaler", StandardScaler()) # Usaremos el StandarScaler, el mas común
])

In [114]:
# Pipeline para atributos categoricos

cat_pipeline = Pipeline([
    # Sin valores faltantes
    ("cat_encoder", OneHotEncoder(sparse_output=False)) # Usaremos OneHotEnconder
])

In [115]:
# Pipeline con ordinal enconder para datos con pensamientos suicidas registrados, debido a que 
# según el EDA y logica del mundo real las personas con esto tienden a tener depresión, por lo
# tanto se añade una ligera preferencia a los que han tenido
catS_pipeline = Pipeline([
    ("encoder", OrdinalEncoder())
])

### Pipeline completo

In [116]:
full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", cat_pipeline, cat_attribs),
    ("catS", catS_pipeline, cat_attribs_st)
])

### Separacion de datos

Separaremos los datos para entrenar y para los tests

**Entrenamiento:**

In [117]:
X_train = full_pipeline.fit_transform(train_data)

In [118]:
X_train.shape

(22320, 18)

In [119]:
X_train[0,:]

array([ 0.84794271,  0.39027391, -1.92862411,  0.        ,  1.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ])

In [120]:
y_train = train_data["Depression"]
y_train

283      0
6919     1
27325    0
14732    1
24938    0
        ..
9828     1
25575    1
21567    0
26909    1
3635     1
Name: Depression, Length: 22320, dtype: int64

**Pruebas:**

In [121]:
X_test = full_pipeline.transform(test_data)
X_test

array([[ 0.64404401,  0.39707967,  0.76439856, ...,  0.        ,
         1.        ,  0.        ],
       [-0.98714559, -0.84837396,  0.22579402, ...,  1.        ,
         0.        ,  0.        ],
       [-0.17155079, -1.69909365, -0.31281051, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-1.39494299,  0.66930997,  0.49509629, ...,  1.        ,
         0.        ,  1.        ],
       [-1.19104429,  0.58764088, -0.85141504, ...,  0.        ,
         1.        ,  1.        ],
       [-1.19104429,  1.07765542, -0.58211278, ...,  1.        ,
         0.        ,  1.        ]], shape=(5581, 18))

In [122]:
X_test.shape

(5581, 18)

In [123]:
y_test = test_data["Depression"]
y_test

21469    1
23715    0
21552    1
14534    0
13724    0
        ..
24278    0
6852     1
14656    1
13054    0
27139    1
Name: Depression, Length: 5581, dtype: int64

## Creación y entrenamiento de las redes neuronales

In [146]:
model_configs = [
    {"activation": "logistic", "solver": "adam", "hidden_layer_sizes": (5,)},
    {"activation": "logistic", "solver": "adam", "hidden_layer_sizes": (15,5)},
    {"activation": "relu", "solver": "lbfgs", "hidden_layer_sizes": (28, 10, 5)},
    {"activation": "tanh", "solver": "adam", "hidden_layer_sizes": (40,)},
    {"activation": "relu", "solver": "adam", "hidden_layer_sizes": (40, 15)},
]

In [147]:
model_list = []

for i, config in enumerate(model_configs, 1):
    model = MLPClassifier(**config, alpha=1e-5, random_state=123)
    model.fit(X_train, y_train)
    model_list.append(model)
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring="accuracy")
    print(f"Modelo {i}: Accuracy promedio = {scores.mean():.4f}")

Modelo 1: Accuracy promedio = 0.7846
Modelo 2: Accuracy promedio = 0.7852
Modelo 3: Accuracy promedio = 0.7818
Modelo 4: Accuracy promedio = 0.7848
Modelo 5: Accuracy promedio = 0.7790


## Analizando desempeño en el conjunto de prueba

In [149]:
y_pred_list = [] 
for i, model in enumerate(model_list, 1):
    y_pred = model.predict(X_test)
    y_pred_list.append(y_pred)

### Calculando la precisión y matriz de confusión

In [150]:
for i, y_pred in enumerate(y_pred_list, 1):
    accs = accuracy_score(y_test, y_pred)
    print(f"Modelo {i}: Accuracy Score = {accs:.4f}")

Modelo 1: Accuracy Score = 0.7850
Modelo 2: Accuracy Score = 0.7839
Modelo 3: Accuracy Score = 0.7839
Modelo 4: Accuracy Score = 0.7827
Modelo 5: Accuracy Score = 0.7789


In [151]:
for i, y_pred in enumerate(y_pred_list, 1):
    conmatrix = confusion_matrix(y_test, y_pred)
    print(f"Modelo {i}: Matriz de confusión = {conmatrix}")

Modelo 1: Matriz de confusión = [[1581  718]
 [ 482 2800]]
Modelo 2: Matriz de confusión = [[1582  717]
 [ 489 2793]]
Modelo 3: Matriz de confusión = [[1579  720]
 [ 486 2796]]
Modelo 4: Matriz de confusión = [[1610  689]
 [ 524 2758]]
Modelo 5: Matriz de confusión = [[1548  751]
 [ 483 2799]]


El mejor modelo con una precisión del 78.25% con datos de prueba tiene los siguientes hiperparametros:

- Funcion de activación: logistic
- optimizador: adam
- capas ocultas: 15,5

## Variación de hiperparametros

A continuacion crearemos dos modelos adicionales con los parametros base del modelo que obtuvo la mejor precision con el objetivo de variar un hiperparametro y examinar si hay mejoria, el hiperparametro a variar es uno muy conocido en las redes neuronales muy relacionado al solver, la taza de aprendizaje inicial la cual es utilizada por los optimizadores que actualizan pesos iterativamente.

Un valor muy alto puede hacer que el modelo no converja (salte soluciones óptimas) y un valor muy bajo puede hacer que tarde mucho en converger o quede atascado en mínimos locales. Por lo que vale la pena investigarlo

In [152]:
model_variation = [
    {"activation": "logistic", "solver": "adam", "hidden_layer_sizes": (5,), "learning_rate_init": 0.0001},
    {"activation": "logistic", "solver": "adam", "hidden_layer_sizes": (5,), "learning_rate_init": 0.1},
]

In [153]:
model_variation_list = []

for i, config in enumerate(model_variation, 1):
    model = MLPClassifier(**config, alpha=1e-5, random_state=123)
    model.fit(X_train, y_train)
    model_variation_list.append(model)
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring="accuracy")
    print(f"Modelo {i}: Accuracy promedio = {scores.mean():.4f}")

Modelo 1: Accuracy promedio = 0.7819
Modelo 2: Accuracy promedio = 0.7818


In [145]:
# Evaluamos cada modelo sobre X_test y almacenamos el accuracy
results_variation = []
for idx, model in enumerate(model_variation_list, 1):
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results_variation.append({"Modelo": f"Modelo {idx}", "Accuracy": round(acc,4)})

# Convertimos a DataFrame y ordenamos de mayor a menor accuracy
df_results = pd.DataFrame(results_variation).sort_values(by="Accuracy", ascending=False).reset_index(drop=True)

# Mostramos la tabla
print(df_results)

     Modelo  Accuracy
0  Modelo 1    0.7875
1  Modelo 2    0.7852


## Analisis 

Podemos observar cómo el modelo con una tasa mucho menor a la por defecto (learning_rate_init = 0.0001) tiene un mejor comportamiento, aunque la mejora en el accuracy es ligera. Esto indica que, para esta arquitectura y conjunto de datos, un aprendizaje más lento permite una convergencia más estable. Por otro lado, al incrementar la tasa a 0.1, el rendimiento disminuyó ligeramente, lo cual sugiere que una tasa de aprendizaje demasiado alta puede afectar negativamente la capacidad del modelo para ajustarse correctamente a los datos.

En general, se concluye que el valor por defecto (0.001) ya ofrece un buen equilibrio, pero reducirlo puede ser una opción válida para optimizar la precisión en tareas sensibles como la predicción de síntomas depresivos. Aun así, el impacto observado no fue significativo, lo que evidencia que, bajo un preprocesamiento adecuado y una red simple, el modelo es bastante robusto frente a pequeñas variaciones de este hiperparámetro.

